In [ ]:
library('purrr')
library('tidyverse')

Consider this example from [Think Stats 2e](https://greenteapress.com/wp/think-stats-2e/): 

You operate a casino and a player at one of your craps tables is winning. You suspect the player has employed
a crooked die. You collect the following data for the players first 60 rolls with the die in question.

In [ ]:
observed_data = tribble(
    ~die_roll, ~count,
     1,          8,
     2,          9,
     3,         19,
     4,          5,
     5,          8, 
     6,          11
)

observed_data

Let's select a "test statistic" quantify how the observed data deviates from what we would expect *on average* from a fair die. Then we can find the null distribution of the test statistic and leverage that distribution to investigate the fairness of the suspicious die.

The following is a table that represents a fair die:

In [ ]:
fair_die = tribble(
    ~die_roll,
    1,
    2,
    3,
    4,
    5, 
    6
)

fair_die

We can use the `slice_sample` with `replace = TRUE` to simulate a "n" dice rolls.

In [ ]:
slice_sample(fair_die, n = 10, replace = TRUE)

The following is a function that calcuates the absolute deviance for a set of "n" rolls from what we would expect from a fair dice *on average*.

```r
abs_deviance = function(roll_frequencies) {
    n = sum(roll_frequencies)
    expected = n / 6
    abs(roll_frequencies - expected) |> sum()
}

```

Create this function in the cell below and calculate "absolute deviance" for the suspicious die.

(**Hint:** use `summarize(abs_dev = abs_deviance(count))`)

The `purrr` code below generates simulates rolling a fair die 60 times for 1,000 iterations.

```r
n_iterations = 1000
n_rolls = 60

1:n_iterations |> map_dfr(~slice_sample(fair_die, n = 60, replace = TRUE), .id = 'replicate')

```

Run this code and collect the output in a table called `simulated_data`.

Now we are ready to calculate the null distribution of our test statistic!

Using `simulated_data`,
1. `group_by` the `replicate` variable (add `.drop = FALSE` to make sure you include zero frequency counts),
2. `count` the frequency of `die_roll`. Last, 
3. `summarize` each group using the `abs_deviance` function from above. Call you new column `abs_dev`.

Capture the output into a table called `abs_dev_null_dist`.

Plot a histogram of `abs_dev`. Use `geom_vline` to show the positon of our the test statistic calculated from the suspicious die.

Can you calculate a "p-value" for our suspicous die using the null distribution and the observed test statistic?

Redo the steps above but this time use the [Chi square](https://en.wikipedia.org/wiki/Chi-squared_test) statistic. The **Chi square** test statistic simply quantifies the sum of squared differences from expected values divided by the expected value.

You can re-use `simulated_data`.

Here is a function in `R`:

```r
chi_square = function(roll_frequencies) {
    n = sum(roll_frequencies)
    expected = n / 6
    sum((roll_frequencies - expected)**2 / expected)
}

```

Is the "p-value" using the Chi square test statistic lower or higher than that calcuated using "absolute deviance" as the test statistic? 